In [0]:
import pyspark.sql.functions as F


df_silver = spark.table("cvee.jobs_silver")
df_silver=df_silver.drop("vector_text_input")
json_cols = ["lieuTravail", "entreprise", "contact", "agence",
             "origineOffre", "contexteTravail",
             "salaire", "competences", "formations", "langues",
             "permis", "qualitesProfessionnelles"]
for col in json_cols:
    if col in df_silver.columns:
        df_silver = df_silver.withColumn(col, F.to_json(F.col(col)))

max_ingestion_date = df_silver.agg(F.max("ingestion_date")).collect()[0][0]
df_silver = df_silver.filter(F.col("ingestion_date") == max_ingestion_date)



#Export silver table
df_silver.write.mode("overwrite").parquet("s3a://cvee-bucket-eu-north-1/jobs_silver")


#display
num_rows = df_silver.count()
column_names = df_silver.columns
print(f"Number of lines sent : {num_rows}")
print(f"Number of columns : {len(column_names)}")
print(f"Columns : {column_names}")

In [0]:
df_gold = spark.table("cvee.jobs_gold")
df_gold = df_gold.withColumn("embedding", F.to_json(F.col("embedding")))
df_gold = df_gold.filter(F.col("ingestion_date") == max_ingestion_date)

#Export gold table
df_gold.write.mode("overwrite").parquet("s3a://cvee-bucket-eu-north-1/jobs_gold")

#display
num_rows = df_gold.count()
column_names = df_gold.columns
print(f"Number of lines sent : {num_rows}")
print(f"Number of columns : {len(column_names)}")
print(f"Columns : {column_names}")